# CIFAR-10 pre-processing in TensorFlow

TensorFlow offers some useful function to pre-process images https://www.tensorflow.org/api_guides/python/image#Image_Adjustments



Also some functions for denoising are available https://www.tensorflow.org/api_guides/python/image#Denoising

The advantage is that these functions are GPU-ready

## TF image pre-processing on CIFAR-10

In [1]:
# general imports 
import tensorflow as tf 


In [2]:
# importing keras dataset 
from keras.datasets import cifar10 # standard cifar-10 plain dataset, not pre-scaled

Using TensorFlow backend.


In [3]:
# unpack train and test set 
train_set , test_set = cifar10.load_data() 
x_train, y_train = train_set 
x_test, y_test = test_set 

In [4]:
# check data shape 
print(x_test.shape)
print(y_test.shape)

(10000, 32, 32, 3)
(10000, 1)


For pre-processing we are nly interested in the x_test and x_train instances and not in the labels


It is actually possible to process the images using the standard see.run and the feed_dict but this is sub-optimal according to TF creators. 

There is actually a better way which is more efficient wnd is based on using queues. 
https://kratzert.github.io/2017/06/15/example-of-tensorflows-new-input-pipeline.html

In [12]:
# create TensorFlow Dataset objects from train and test set 
from tensorflow.contrib.data import Dataset, Iterator

# first the data which has to be processed must be converted in a tf dataset object 
training_data = Dataset.from_tensor_slices((x_train,y_train))


In [6]:
# create a tensorflow iterator object 
iterator = Iterator.from_structure(training_data.output_types,
                                   training_data.output_shapes) 

next_element = iterator.get_next()

# create an initializer for the dataset transf operation 
training_init_op = iterator.make_initializer(training_data)

## Inefficient code 

In [7]:
with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(training_init_op)

    # get each element of the training dataset until the end is reached 
    while True:
        try:
            image,label = sess.run(next_element)
            # processing goes here 
            tf.image.per_image_standardization(image)
            print(label)
            
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break


[6]
[9]
[9]
[4]
[1]
[1]
[2]
[7]
[8]
[3]
[4]
[7]
[7]
[2]
[9]
[9]
[9]
[3]
[2]
[6]
[4]
[3]
[6]
[6]
[2]
[6]
[3]
[5]
[4]
[0]
[0]
[9]
[1]
[3]
[4]
[0]
[3]
[7]
[3]
[3]
[5]
[2]
[2]
[7]
[1]
[1]
[1]
[2]
[2]
[0]
[9]
[5]
[7]
[9]
[2]
[2]
[5]
[2]
[4]
[3]
[1]
[1]
[8]
[2]
[1]
[1]
[4]
[9]
[7]
[8]
[5]
[9]
[6]
[7]
[3]
[1]
[9]
[0]
[3]
[1]
[3]
[5]
[4]
[5]
[7]
[7]
[4]
[7]
[9]
[4]
[2]
[3]
[8]
[0]
[1]
[6]
[1]
[1]
[4]
[1]
[8]
[3]
[9]
[6]
[6]
[1]
[8]
[5]
[2]
[9]
[9]
[8]
[1]
[7]
[7]
[0]
[0]
[6]
[9]
[1]
[2]
[2]
[9]
[2]
[6]
[6]
[1]
[9]
[5]
[0]
[4]
[7]
[6]
[7]
[1]
[8]
[1]
[1]
[2]
[8]
[1]
[3]
[3]
[6]
[2]
[4]
[9]
[9]
[5]
[4]
[3]
[6]
[7]
[4]
[6]
[8]
[5]
[5]
[4]
[3]
[1]
[8]
[4]
[7]
[6]
[0]
[9]
[5]
[1]
[3]
[8]
[2]
[7]
[5]
[3]
[4]
[1]
[5]
[7]
[0]
[4]
[7]
[5]
[5]
[1]
[0]
[9]
[6]
[9]
[0]
[8]
[7]
[8]
[8]
[2]
[5]
[2]
[3]
[5]
[0]
[6]
[1]
[9]
[3]
[6]
[9]
[1]
[3]
[9]
[6]
[6]
[7]
[1]
[0]
[9]
[5]
[8]
[5]
[2]
[9]
[0]
[8]
[8]
[0]
[6]
[9]
[1]
[1]
[6]
[3]
[7]
[6]
[6]
[0]
[6]
[6]
[1]
[7]
[1]
[5]
[8]
[3]
[6]
[6]
[8]
[6]
[8]
[4]
[6]
[6]


KeyboardInterrupt: 

## Always work with batches iterator is bad 

In [19]:
# An iterator object will be created to GET BATCHES OF IMAGES AND NOT SINGLE IMAGES 
batch_size = 10000

batch = training_data.batch(batch_size)

iterator = batch.make_one_shot_iterator()

next_element = iterator.get_next()


training_init_op = iterator.make_initializer(batch)

processed_image = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), tf.cast(next_element[0],'float32'))



# see https://www.tensorflow.org/programmers_guide/datasets#simple_batching


## Following code is still inefficient but is a TF fault as tf.image methods does not allow directly processing of batches

In [20]:
import time 

In [21]:
next_element[0]

<tf.Tensor 'IteratorGetNext_2:0' shape=(?, 32, 32, 3) dtype=uint8>

In [22]:
t0 = time.time()

with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(training_init_op)

    # get each element of the training dataset until the end is reached 
    while True:
        try:
            std_image=sess.run(processed_image)
            # processing goes here 
            
            
            
            
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break

print('Total time employed to process dataset: %.5f seconds' % (time.time() - t0))        

End of training dataset.
Total time employed to process dataset: 4.47042 seconds


In [22]:
# test if the images have been processed 
std_image



<tf.Tensor 'map_11/TensorArrayStack/TensorArrayGatherV3:0' shape=(10000, 32, 32, 3) dtype=uint8>

In [19]:
with tf.Session() as sess:
    nump=sess.run(std_image)

InvalidArgumentError: TensorArray dtype is uint8 but Op is trying to write dtype float.
	 [[Node: map_5/while/TensorArrayWrite/TensorArrayWriteV3 = TensorArrayWriteV3[T=DT_FLOAT, _class=["loc:@map_5/while/div"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](map_5/while/TensorArrayWrite/TensorArrayWriteV3/Enter, map_5/while/Identity/_19, map_5/while/div, map_5/while/Switch_1/_27, ^_cloopmap_5/while/TensorArrayReadV3/_1)]]
	 [[Node: map_5/while/TensorArrayWrite/TensorArrayWriteV3/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_57_map_5/while/TensorArrayWrite/TensorArrayWriteV3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](^_cloopmap_5/while/Cast/_6)]]

Caused by op 'map_5/while/TensorArrayWrite/TensorArrayWriteV3', defined at:
  File "/home/sam/anaconda3/lib/python3.6/runpy.py", line 193, in _